In [27]:
#Projeto Final de Mineração De Dados
# Este trabalho está associado ao projeto de mobilidade
#Início da Exploração de Dados

############################################################

### Intalando Pacotes
#Função instalar pacotes
InstalaPacotes = function(pacote){
  if (pacote %in% rownames(installed.packages())){
    mensagem <- paste("Pacote", pacote, "Já Instalado")
  }else{
    mensagem <- paste("Instalando Pacote", pacote)
    install.packages(pacote)  
  }
  return(mensagem)  
}


#Instala Pacotes Necessários
InstalaPacotes("dplyr") 
InstalaPacotes("plyr")
InstalaPacotes("reshape2")
#InstalaPacotes("readr")
#InstalaPacotes("stringr")
#InstalaPacotes("stringi")
InstalaPacotes("sqldf")
#InstalaPacotes("microbenchmark")
#InstalaPacotes("mgcv")
InstalaPacotes("ggplot2")
InstalaPacotes("proto") #Adicionado para executar sqldf no Jupyter
InstalaPacotes("RSQLite")
#InstalaPacotes("gsubfn") #Adicionado para executar sqldf no Jupyter
#InstalaPacotes("chron") #Adicionado para executar sqldf no Jupyter
InstalaPacotes("xtable") #Exportar para o Latex

#Carrega Bibliotecas:
library('dplyr')
library("plyr")
library('reshape2')
#library('readr')
#library('stringr')
#library('stringi')
library('sqldf')
#library('microbenchmark')
#library('mgcv')
library('ggplot2')
library('proto')
library('RSQLite')
library('xtable')

[1] "Pacote dplyr Já Instalado"

[1] "Pacote plyr Já Instalado"

[1] "Pacote reshape2 Já Instalado"

[1] "Pacote sqldf Já Instalado"

[1] "Pacote ggplot2 Já Instalado"

[1] "Pacote proto Já Instalado"

[1] "Pacote RSQLite Já Instalado"

[1] "Pacote xtable Já Instalado"

------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: 'plyr'

The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



Importando Arquivo do github

In [43]:
### As bases agregadas estão em:
#"https://github.com/febsrj/CEFET-Mobilidade-AnlExpl/tree/master"
#knitr::knit(text ='```{r}
#load(url("https://github.com/febsrj/CEFET-Mobilidade-AnlExpl/tree/master?raw=TRUE"))
#```')

#files <- list.files(list.files(path="https://github.com/febsrj/CEFET-Mobilidade-AnlExpl/tree/master" pattern="*.RData", full.names=TRUE, recursive=FALSE)) for(i in files){ print(i) df <- load(i) assign(gsub("\\..*","", i), df ) #extracts the string before period rm(df) }

Aggdata <- NULL
media <- NULL
mediaquad <- NULL
for(i in 1:126){
        knitr::knit(text ='{r}
      Aggdatai<-get(load(url("https://github.com/febsrj/CEFET-Mobilidade-AnlExpl/tree/master" pattern="*.RData", full.names=TRUE, recursive=FALSE")))
      mediai<-aggregate(velocidade ~ dt,Aggdatai,mean)
      mediaquadi<-aggregate(velocidade ~ dt+estacao,Aggdatai,mean)
      cat(i,"\n")
      ')
      }


#https://github.com/febsrj/CEFET-Mobilidade-AnlExpl/tree/master
#input <- dir(pattern='.RData')
#L <- length(input)


In [44]:
df <- ldply(Aggdata, data.frame) 
dfmediadia<-ldply(media, data.frame) 
dfmediaquad<-ldply(mediaquad, data.frame) 


In [46]:
####Limpando as linhas duplicadas q contém zero

#verificando se há linha duplicada
#sqldf("select dt, count(dt) as conta from dfmediadia group by dt having conta > 1")

#deletando as linhas duplicadas
linhasdelete<-c(21,120)
dfmediadianew<-dfmediadia[-linhasdelete,]


In [47]:
# Ordenação da base de dados - ordem crescente (default)
#dfmediadia_orddt <- dfmediadianew[order(dfmediadia$dt),]
#plot(dfmediadianew, type="o", col="darkblue")

head(dfmediadianew,10)
tail(dfmediadianew,10)

#Fazendo uma série temporal para cada quadrante
pivotquad<-dcast(dfmediaquad,dt~estacao,value.var="velocidade",mean)



<0 x 0 matrix>

<0 x 0 matrix>

ERROR: Error: value.var (velocidade) not found in input


In [48]:
# quantos quadrantes existem?
quad<-dfmediaquad[,2]
conta<-unique(quad)
length(conta)


ERROR: Error in `[.data.frame`(dfmediaquad, , 2): undefined columns selected


In [49]:
#Fazendo as estatísticas
sumario<-sqldf("select 
               estacao,
               avg(velocidade) as media,
               min(velocidade) as minimo,
               max(velocidade) as maximo,
               stdev(velocidade) as desvpad
               from dfmediaquad 
               group by estacao
               ")


ERROR: Error in (function (classes, fdef, mtable) : unable to find an inherited method for function 'dbQuoteIdentifier' for signature '"SQLiteConnection", "NULL"'


In [50]:
# Ordenação da base de dados - ordem crescente (default)
sumario_ord <- sumario[order(sumario$media),]

head(sumario_ord, 10)

#Fazendo a correlação dos dados
m.corr<-cor(pivotquad[,2:1811],use ="pairwise.complete.obs")

#fazendo unpivot table
melt_m.corr<-melt(m.corr)

# Ordenação da base de dados - ordem crescente (default)
melt_m.corr <- melt_m.corr[order(melt_m.corr$value,decreasing=TRUE),]
maior_corr<-melt_m.corr[melt_m.corr$value<1,c("Var1","Var2","value")]


ERROR: Error in eval(expr, envir, enclos): objeto 'sumario' não encontrado
